In [2]:
# подключить библиотеку для математических операций
import math
from time import time

import plotly.express as px

### Как НЕ надо читать SPSS файлы питоном

In [8]:
# импортируем библиотеку для работы с таблицами
import pandas as pd

In [10]:
# в Пандас есть метод (функция) для чтения СПСС-файла, pd.read_spss, но она не рекомендуется к использованию
t1 = time() # запоминаем текущее время в сточностью до секунды
data = pd.read_spss('IND_KOUZH_2018.sav')
# возьмем ещё раз текущее время с точностью до секунды и вычтем то время, которое было зафиксировано ранее;
# таким образом, получим время, которое израсходуется на исполнение кода.
time() - t1 # время в секундах.

222.13054013252258

In [4]:
# по умолчанию, выводится командой пять строк; строчки заголовков по умолчанию не входят в число строк.
# файлы росстата в вин-кодировках, а питон использует кодировку UTF-8, это кодировка линя-мака.
data.head()

,H01_01,H01_02,H01_03,H01_04,H01_05,H01_06,H01_07,H01_08,H01_09,H01_10,...,OTP_DET_OF,OTP_REB_OF,HAVE_D_1_5,HAVE_D_1_5_3,ROD_ALK,DX,V626,GL_DH,GL_DH_VOZR,GL_DH_SUPR
0,Æåíñêèé,79.0,Îòâåòñòâåííîå ëèöî,Âäîâåö/âäîâà,NaN,NaN,ïîñòîÿííî ïðîæèâàåò,Äà,NaN,Äà,...,NaN,NaN,0.0,0.0,NaN,1.0,1.0,1.0,ãëàâà ÄÕ ñòàðøå òðóäîñïîñîáíîãî âîçðàñòà,NaN
1,Æåíñêèé,72.0,Îòâåòñòâåííîå ëèöî,Âäîâåö/âäîâà,NaN,NaN,ïîñòîÿííî ïðîæèâàåò,Äà,NaN,Äà,...,NaN,NaN,0.0,0.0,NaN,1.0,NaN,1.0,ãëàâà ÄÕ ñòàðøå òðóäîñïîñîáíîãî âîçðàñòà,NaN
2,Æåíñêèé,89.0,Îòâåòñòâåííîå ëèöî,Âäîâåö/âäîâà,NaN,NaN,ïîñòîÿííî ïðîæèâàåò,Äà,NaN,Äà,...,NaN,NaN,0.0,0.0,NaN,1.0,1.0,1.0,ãëàâà ÄÕ ñòàðøå òðóäîñïîñîáíîãî âîçðàñòà,NaN
3,Ìóæñêîé,25.0,Îòâåòñòâåííîå ëèöî,Íèêîãäà íå ñîñòîÿë(à) â áðàêå,NaN,NaN,ïîñòîÿííî ïðîæèâàåò,Äà,NaN,Äà,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.0,ãëàâà ÄÕ òðóäîñïîñîáíîãî âîçðàñòà,NaN
4,Ìóæñêîé,26.0,Îòâåòñòâåííîå ëèöî,Íèêîãäà íå ñîñòîÿë(à) â áðàêå,NaN,NaN,ïîñòîÿííî ïðîæèâàåò,Äà,NaN,Äà,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.0,ãëàâà ÄÕ òðóäîñïîñîáíîãî âîçðàñòà,NaN


Проблема pandas-а: чтение большого файла (этот - 99.1 МБ) выполняется в одном процессе (см., например, https://towardsdatascience.com/multithreading-multiprocessing-python-180d0975ab29 ) - читается долго (минуту подождать можно, но что, если у нас 60 таких файлов), нельзя передать параметр, в какой кодировке читать строковые значения. 

### Как надо читать SPSS файлы питоном

In [11]:
# импортируем другую библиотеку pyreadstat, которая оринетирована на работу с форматами статистических (в т.ч., платных) программ.
# ! pip (установка) install pyreadstat==1.2.0
import pyreadstat

In [12]:

t1 = time()
fpath = 'IND_KOUZH_2018.sav'
# метод называется иначе - мультипроцессинг, и у него иной набор параметров;
# параметры (аргументы):
# 1. неименованный (позиционный) + обязательный = аргумент pyreadstat.read_sav: покахывает нам, какой тип файла читать.
# 2. неименованный (позиционный) + обязательный = аргумент : путь к файлу (у меня это на диске С, где "лежит" ноутбук).
# 3. именованный (непозиционный) + необязательный = аргумент: сколько процессов на чтение файла запускать операционной системе.
# 4. именованный (непозиционный) + необязательный = аргумент: из какой кодировки прочитать исходный файл.
# 5. тут мы указываем ДВЕ переменные, куда мы записываем результат исполнения кода.
# 6. если тут мы укажем одну переменную, то работать команда не будет.
df, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath, num_processes=16, encoding='cp1251') 
time() - t1

15.655770063400269

In [6]:
# 1. вывести первые пять строчек (head): в памяти лежит объект DF, функция head возвращает первые пять строчек.
# 2. про функции можно тут думать так, как мы думаем в математике - мы получаем некий расчет условной части;
# который мы можем далее перееиспользовать; т.е., это как в Эксель, когда мы посчитали одну ячейку и можем далее ссылаться на нее.
# 3. как мы видим, что изменилась не только кодировка (все прочиталось), но и стали адекватно прочитываться форматы,
# которые залолжены в файлике sav (IBM SPSS).
# 4. Можно посмотреть (для общего развития), как это выглядит в IBM SPSS (социологи часто работают именно в этом приложении).
df.head()

,H01_01,H01_02,H01_03,H01_04,H01_05,H01_06,H01_07,H01_08,H01_09,H01_10,...,OTP_DET_OF,OTP_REB_OF,HAVE_D_1_5,HAVE_D_1_5_3,ROD_ALK,DX,V626,GL_DH,GL_DH_VOZR,GL_DH_SUPR
0,2.0,79.0,0.0,3.0,NaN,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,0.0,0.0,NaN,1.0,1.0,1.0,2.0,NaN
1,2.0,72.0,0.0,3.0,NaN,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,0.0,0.0,NaN,1.0,NaN,1.0,2.0,NaN
2,2.0,89.0,0.0,3.0,NaN,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,0.0,0.0,NaN,1.0,1.0,1.0,2.0,NaN
3,1.0,25.0,0.0,6.0,NaN,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.0,1.0,NaN
4,1.0,26.0,0.0,6.0,NaN,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.0,1.0,NaN


In [13]:
# команда dir показывает, какие есть атрибуты у объекта; проверим атрибуты meta - выведем список.
dir(meta)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'column_labels',
 'column_names',
 'column_names_to_labels',
 'file_encoding',
 'file_format',
 'file_label',
 'missing_ranges',
 'missing_user_values',
 'notes',
 'number_columns',
 'number_rows',
 'original_variable_types',
 'readstat_variable_types',
 'table_name',
 'value_labels',
 'variable_alignment',
 'variable_display_width',
 'variable_measure',
 'variable_storage_width',
 'variable_to_label',
 'variable_value_labels']

In [14]:
# 1. в meta - объект - записались наши метаданные, как это в приложении SPSS на отдельной странице.
# 2. у питоновских объектов есть атрибуты; если мы хотим посмотреть, что лежит именно в атрибутах объекта - которые мы вывели выше
# то нам нужно обратиться к нему через точку.
meta.column_names_to_labels

{'H01_01': 'Пол',
 'H01_02': 'Число исполнившихся лет',
 'H01_03': 'Родственное отношение',
 'H01_04': 'Семейное положение',
 'H01_05': 'C какого времени семейное положение такое',
 'H01_06': 'Индивидуальный код супруга',
 'H01_07': 'Проживание в домохозяйстве',
 'H01_08': 'Проживание на день опроса',
 'H01_09': 'Причина отсутствия на момент опроса',
 'H01_10': 'Возможность дать информацию о самом себе',
 'H01_11': 'Причина, по которой респондент не может дать сам о себе информацию',
 'H01_12': 'Код лица, ответившего за респондента',
 'H01_13': 'Тип семейной единицы',
 'H01_14': 'Порядковый номер семейной единицы в домохозяйстве',
 'I00_12': 'Час начала опроса',
 'I00_13': 'Минуты начала опроса',
 'I00_14': 'Час окончания опроса',
 'I00_15': 'Минуты окончания опроса',
 'I01_01': 'Семейное положение',
 'I01_02': 'Проживание супругов',
 'I01_03': 'Причина раздельного проживания супругов',
 'I01_04': 'Наличие детей проживающих отдельно',
 'I01_05_01': 'Дети проживают в том же населенном п

In [18]:
# "название переменной (названия колонок, как они даны в файле sav, в метку (labels - служебные названия, чтобы было проще к ним образаться);
# т.е. можно по ключу получить значение, но невозможно получить по значению ключ.
# cтруктура данных из ключей и значений, называется словарем.
# в фигурных скобках пишется до двоеточия ключ, а после двоеточия - значение.
meta.variable_to_label

{'H01_01': 'labels0',
 'H01_03': 'labels1',
 'H01_04': 'labels2',
 'H01_07': 'labels3',
 'H01_08': 'labels4',
 'H01_09': 'labels5',
 'H01_10': 'labels4',
 'H01_11': 'labels6',
 'H01_13': 'labels7',
 'I01_01': 'labels8',
 'I01_02': 'labels9',
 'I01_03': 'labels10',
 'I01_04': 'labels11',
 'I01_05_01': 'labels12',
 'I01_05_02': 'labels13',
 'I01_05_03': 'labels14',
 'I01_05_04': 'labels15',
 'I01_05_05': 'labels16',
 'I01_06_03': 'labels17',
 'I01_07_01': 'labels18',
 'I01_07_02': 'labels18',
 'I01_07_03': 'labels18',
 'I01_07_04': 'labels18',
 'I01_07_05': 'labels19',
 'I01_08_01': 'labels20',
 'I01_08_02': 'labels21',
 'I01_08_03': 'labels22',
 'I01_08_04': 'labels23',
 'I01_08_05': 'labels24',
 'I01_08_06': 'labels25',
 'I01_09_01': 'labels26',
 'I01_09_02': 'labels27',
 'I01_09_03': 'labels28',
 'I01_09_04': 'labels29',
 'I01_09_05': 'labels30',
 'I01_09_06': 'labels31',
 'I01_09_07': 'labels32',
 'I01_09_08': 'labels33',
 'I01_09_09': 'labels34',
 'I01_09_10': 'labels35',
 'I01_09_1

In [22]:
# 1. следующей командой, посмотрим значения меток (в IBM SPSS они будут у нас выпадающим списком на листе "представление переменных" -->
# --> значения меток)
# 2. тут мы видим, что у нас представление данных происходит по "значениям словаря", которые мы можем получить по "ключу" 
# словаря; т.е., для value_labels это уже ключ, а для словаря - "значение".
meta.value_labels

{'labels0': {1.0: 'Мужской', 2.0: 'Женский'},
 'labels1': {0.0: 'Ответственное лицо',
  1.0: 'Жена/Муж',
  2.0: 'Сын/Дочь',
  3.0: 'Зять/Невестка',
  4.0: 'Отец/Мать',
  5.0: 'Родители жены/мужа',
  6.0: 'Брат/сестра',
  7.0: 'Внук/внучка',
  8.0: 'Племянник/ца',
  9.0: 'Другие родственники',
  10.0: 'Не родственники'},
 'labels2': {1.0: 'Состоит в зарегистрированном браке',
  2.0: 'Состоит в незарегистрированном браке',
  3.0: 'Вдовец/вдова',
  4.0: 'Разведен (а)',
  5.0: 'Разошелся (лась)',
  6.0: 'Никогда не состоял(а) в браке'},
 'labels3': {1.0: 'постоянно проживает',
  2.0: 'регулярно отсутствует в связи с работой, учебой в другом нас',
  3.0: 'отсутствует более 12 месяцев'},
 'labels4': {1.0: 'Да', 2.0: 'Нет'},
 'labels5': {1.0: 'отъезд на заработки',
  2.0: 'отъезд на учебу',
  3.0: 'отъезд в командировку, в дальнее плавание, призыв на военно-',
  4.0: 'отъезд на отдых (на дачу), по семейным и личным обстоятельст',
  5.0: 'госпитализация',
  6.0: 'служба по призыву или контракт

In [23]:
# мы хотим выбрать только определенные колонки из dataframe, для этого мы заводим список их названий.
# 
selection = ['DEN_NA_DUSHU', 'H00_02', 'H00_04', 'R_1', 'R_1_1']

In [27]:
# чтобы получить новую переменную, мы хотим, чтобы у нас была таблица (датафрейм), только с теми колонками, которые мы "хотим".
# фактически, это ускоряет нам вычисления. с другой стороны, можно пользоваться и исходной таблицей.
selected = df[selection]
# без пустых скобочек - атрибут; если со скобочками (пустыми или не пустыми), то это метод.
# метод также является атрибутом, потому мы можем его "распечатать", но он у нас не будет исполнен.
selected.head()

,DEN_NA_DUSHU,H00_02,H00_04,R_1,R_1_1
0,17500.0,01,1.0,1.0,1.0
1,12500.0,01,1.0,1.0,1.0
2,12500.0,01,1.0,1.0,1.0
3,17500.0,01,1.0,1.0,1.0
4,27500.0,01,1.0,1.0,1.0


In [29]:
# штука, которая называется "генератор списков", суть его использования в том, что мы хотим переименовать "кошмарные заголвки";
# в человеческие. для этого мы хотим "пройтись" по всем меткам в списке селекшн и получить новый список вместо "селекшн".
# список в данном случае, это вид структуры данных, а в памяти лежит объект класса "список".
selection_names = [meta.column_names_to_labels[label] for label in selection]
# т.е., ранее у нас были в названиях колонок страшные названия из росстата, а теперь у нас наименования из колонки "метки" файла sav.
selected.columns = selection_names

In [34]:
# тут мы распечатали всю таблицу; если бы мы хотели распечатать пять строк, как раньше, то мы вписали бы selected.head().
# со скобочками - метод; без скобочек - объект.
# атрибуты объектов сами тоже являются объектами.
# троеточие появилось у нас, чтобы была возможность вывести табличку "удобоваримо", ибо у нас 130610 строк.
selected

,"Денежный доход в среднем на члена ДХ, в месяц, рублей",Код субъекта РФ,Тип населенного пункта,Место проживания,Размер населенного пункта
0,17500.000000,01,1.0,1.0,1.0
1,12500.000000,01,1.0,1.0,1.0
2,12500.000000,01,1.0,1.0,1.0
3,17500.000000,01,1.0,1.0,1.0
4,27500.000000,01,1.0,1.0,1.0
...,...,...,...,...,...
130605,6428.571429,99,2.0,2.0,9.0
130606,6428.571429,99,2.0,2.0,9.0
130607,6428.571429,99,2.0,2.0,9.0
130608,6428.571429,99,2.0,2.0,9.0


Как отобрать строчки в пандасовом датафрейме, где в колонке какое-то нужное нам значение, с маской: маска - датафрейм, где проставлено, значение ячейки истинно или нет

In [35]:
# 98 - код Республики Саха (Якутии), согласно ОКВЭДам.
# фактически, мы делаем файл разбиения.
# у нас тут есть новый объект - mask (который болтается в оперативной памяти и исчезает из нее, если мы все закроем).
mask = selected['Код субъекта РФ'] == '98'
# распечатать - первые четыре строки - код субъекта; bool - логические данные, булевая логика ("истинно" или "ложно").
print(mask.head())

0    False
1    False
2    False
3    False
4    False
Name: Код субъекта РФ, dtype: bool


In [36]:
# используем только те строчки, где маска True
# когда мы хотим выбрать только те строчки, где код якутии - 98, 
# то мы делаем маску, которая такого же рамера по количеству строчек как датафрейм, но значения в ней булевые (или логические)
# потом этой маской (где код субъекта 98), мы оставим только те строчки, где значение было True.
yakutia = selected[mask]

In [37]:
# тут мы просто вывели на экран посмотреть, и видим, что действительно, осталась только якутия.
yakutia

,"Денежный доход в среднем на члена ДХ, в месяц, рублей",Код субъекта РФ,Тип населенного пункта,Место проживания,Размер населенного пункта
129045,45000.0,98,1.0,1.0,1.0
129046,87500.0,98,1.0,1.0,1.0
129047,55000.0,98,1.0,1.0,1.0
129048,45000.0,98,1.0,1.0,1.0
129049,55000.0,98,1.0,1.0,1.0
...,...,...,...,...,...
130198,6875.0,98,2.0,2.0,10.0
130199,6875.0,98,2.0,2.0,10.0
130200,6875.0,98,2.0,2.0,10.0
130201,6875.0,98,2.0,2.0,10.0


Денежный доход в среднем на члена ДХ, в месяц, рублей - асимметрия, эксцесс, межквартильный размах

Что посчитает метод describe для числовых данных: 

count - количество наблюдений

mean - среднее арифметическое

std - среднеквадратическое отклонение, standard deviation

min - минимальное значение

max - максимальное значение

25%, 50%, 75% - Q1, Q2, Q3

In [40]:
# число наблюдений в табличке, вызываем через точка+Шейп.
# квадратная скобочка, потому как просто шейп вернет объект типа Кортеж (из терминологии Пайтон) - список, который нельзя будет поменять.
# а в квадратных скобках мы берем какой-либо элемент из этого Кортежа - т.е. "0" - первый элемент, "1" - второй элемент, и т.д.
# в Шейпе у нас двумерный датафрейм (таблица). А трехмерных в Пандас (библиотека) и не бывает (возможно). 
# Бывают одномерные, в которых только количество точек (называется такое Сериес - или серия с английского), к которому возможно применить все идентичный методы, что и к датафрейму.
yakutia.shape[0]

1158

In [39]:
# точка+дескрайб - метод, который выводит описательные статистики.
# круглые скобки (пустые) - это означат, что мы имеем дело с методом, а не с атрибутом.
# ниже дана ссылка на документацию Пандас, где есть те методы, которыми вы возможно не очень хорошо владеете.
# тут вы сможете почерпнуть необходимую информацию (при условии определенного упорства).
yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].describe()
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html

count      1158.000000
mean      22951.208981
std       16915.918678
min        2500.000000
25%       11666.666667
50%       17500.000000
75%       28125.000000
max      137500.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64

In [111]:
# точка+апплай - метод, который позволяет применить функцию ко всему сисриесу или датафрейму.
# тут у нас сириес, так как только одна колонка.
# в скобочках у нас один из способов, как возможно задать функцию (возможно это сделать через деф или через лябмбду).
# тут что мы говорим, что после лямбды идет аргумент функции (Х) и после двоеточия, мы указываем то, что с ним делаем.
# у нас есть столбец 'R_1_1' -- есть словать  meta.variable_to_label, в котором есть пары, где ключ - 'R_1_1', 
# значения это лейбл с некрасивым номером (который видели выше).
# потом лейбл с некрасивым номером является ключом в словаре мета.валью.лейблз и из него мы получаем соответствующие значения.
# т.е., это у нас вложенный словарь - словарь в словаре.
# результатом этой конструкции является следующее; система проходит по выделенному столбцу по каждой строке и выводит значение из таблички-словарика-словарика.
# к приемру, у нас цифра два, а во вложенном словарике словарика, под цифрой два такое-то значение.
sizes = yakutia['Размер населенного пункта'].apply(lambda x: meta.value_labels[meta.variable_to_label['R_1_1']][x])
sizes.head()

129045    городской,  менее 50,0 тыс.чел
129046    городской,  менее 50,0 тыс.чел
129047    городской,  менее 50,0 тыс.чел
129048    городской,  менее 50,0 тыс.чел
129049    городской,  менее 50,0 тыс.чел
Name: Размер населенного пункта, dtype: object

Что посчитает метод describe для нечисловых данных:

unique - количество уникальных значений

top - наиболее часто встречающееся значение

freq - частота top-а (мода)

в случае нескольких одинаковых top-ов, какой вывести, выбирается случайно

In [113]:
# 
# 
sizes.describe()

count                               1158
unique                                 6
top       городской,  менее 50,0 тыс.чел
freq                                 448
Name: Размер населенного пункта, dtype: object

In [74]:
# дисперсия (англ. variance)
# что означатет дисперсия, можно посмотреть в лекции.
variance = yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].var()

# распечатать результат.
print(f"Дисперсия c n-1 степенью свободы: {variance}")

# ддоф - число степеней свободы, если не задана явно, то это единица. или иначе, мы получаем исправленную оценку дисперсии 
# число наблюдений минус один.
# тут мы решили показать, насколько могут отличаться результаты, задачи число 
ddof = 10

# ddof - delta degrees of freedom, сколько вычесть из количества наблюдений n чтобы получить степени свободы. по умолчанию 1.
print(f"Дисперсия c n-{ddof} степеней свободы: {yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].var(ddof=ddof)}") 

# находим среднеквадратическое отклонение по сооответствующему параметру.
std = yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].std() 
print(f"Среднеквадратическое отклонение: {std}")

# находим квадрат от сигмы (среднеквадратического  отклонения).
# операция присваивания (присваиваем значение в новую переменную (можно идентично и в существующую); 
# т.е., если переменная ранее уже была создана, то старое значение "пропадет" и запишется новое значение (перезапишется).
variance_from_std = std ** 2 

# распечатаем в консоль результат; фигурные скобки - поставим перед строкой эф - от слова форматирование; 
# и в фигурных скобках прописываем эту переменную.
# f - потому как операция подставить значение из переменной называется форматирование (это из "Си")
print(f"Дисперсия как квадрат среднеквадратического отклонения: {variance_from_std}")


Дисперсия c n-1 степенью свободы: 286148304.7065876
Дисперсия c n-10 степеней свободы: 288391627.6528936
Среднеквадратическое отклонение: 16915.91867758259
Дисперсия как квадрат среднеквадратического отклонения: 286148304.70658755


In [52]:
# мин - среднее арифметическое.
mean = yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].mean()

# round это округление, сколько знаков после запятой
coefficient_of_variation = std / mean

# смотрите выше по поводу применения подобной записи после "принт". "2" - сколько знаков после запятой. 
print(f"Коэффициент вариации: {round(coefficient_of_variation, 2)}")
print(f"Коэффициент вариации: {coefficient_of_variation * 100} %")

Коэффициент вариации: 0.74
Коэффициент вариации: 73.70382401896582 %


In [53]:
# это находим размах, просто математическим действием; выделаем значения макса и мина; круглые скобки, так как это Метод - т.е., мы вычисляем это число.
# это число не лежит в Атрибуте готовое.
R = yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].max() - yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].min()
# распечатку смотри выше.
print(f"Размах вариации: {R}")

Размах вариации: 135000.0


In [ ]:
# идентично тому, что выше, но в отношении Метода поиска средней арифметической.
mean = yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].mean()
# round это округление, сколько знаков после запятой.
# относительный размах вариации (коэффициент осциляции).
scope_of_variation_ = R / mean

Видно, что в методе describe у среднеквадратического отклонения тоже была n-1 степень свободы. Но там выводится меньше знаков после точки.

Немного разные значения дисперсии методами var и квадратом std получились за счет того, что числа с плавающей точкой не могут быть представлены в памяти компьютера как по-настоящему бесконечные (см. https://0.30000000000000004.com/).
Чтобы сравнить, равны ли они, используем math.isclose (см. https://davidamos.dev/the-right-way-to-compare-floats-in-python/ ).

In [54]:
# двумя способами посчитана дисперсия, в одном случае вар, во втором случае - стд квадрат.
# причина различий в том, что различия вызваны операциями с плавающей точкой.
# то, что мы получили true, может говорить нам о том, что в некоемом приближении эти числа - можно считать равными
# в некоемом приближении.
math.isclose(variance, variance_from_std)

True

In [127]:
# тут мы найдем ассиметрию, через функцию .skew() ; можно посчитать по формуле.
yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].skew() 
# см. https://medium.com/@atanudan/kurtosis-skew-function-in-pandas-aa63d72e20de

2.3586097781835695

In [128]:
# найдем эксцесс; можно посчитать по формуле.
# асимметрия и эксцесс также считаются с n-1 степенью свободы; это в Пандас идет автоматически.
yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].kurtosis() 
# см. https://medium.com/@atanudan/kurtosis-skew-function-in-pandas-aa63d72e20de

8.439253026499307

In [70]:
# показать картинку, через фигшоу.
fig = px.box(yakutia, y='Денежный доход в среднем на члена ДХ, в месяц, рублей')
fig.show()

### Средний доход самых бедных 10%

In [78]:
# тут мы задаем Дециль, далее Метод - квантиль;
# в скобочки подставляем значение (надо посмотреть документацию, но очень похоже, что это первый квартиль "по умолчанию",
# или по "дефолту").
decil = yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'].quantile(q=0.10)
poorest = yakutia[yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей'] < decil]

# число наблюдений. Шейп возврашает кортеж. Кортеж - неизменяемый массив элементов.
# из массива можно получать элементы по индексу; т.е., шейп "0" - количество строчек. шейп "1" будет, количество столбцов.
number_of_poorest = poorest.shape[0]
print(f"Количество тех, чей доход меньше дециля ({decil}): {number_of_poorest}")
mean = poorest['Денежный доход в среднем на члена ДХ, в месяц, рублей'].mean()
print(f"Их средний доход: {mean}")

# тут берем сириес (одну колонку).
income = yakutia['Денежный доход в среднем на члена ДХ, в месяц, рублей']
# переменная "средние по децилям" - пустой список. это у нас изменяемый массив элементов 
# (т.е., если бы мы создавали Кортеж, то мы туда не могли бы добавлять элементы; а так как создаем Список, то мы можем туда 
# добавлять элементы). Изменяемый называется ещё "динамический", неизменяемый - "статический".
means_by_deciles = []
# в принципе, тут возможно создать и Кортеж и Список, так как менять мы этот Массив не будем.
# зачем в принципе нужны две разные структуры данных Массива (изменяемая и неизменяемая структуры), кк примеру. если 
# есть большая программа, над которой работают несколько человек, то если один человек задумал, что какой-либо список
# никогда не будет меняться, чтобы другой человек не мог случайно в эту структуру что-либо добавить или удалить.
# и даже один человек может иногда запутаться и случайно изменить что-либо из того, что менять не следует.
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
last_quantile_value = 0
# отступами в четыре пробела (или таб в ноутбуке), обозначаются некие логические блоки (блоки программного кода)
# т.е., то что в этой строчек с отступами.
# следующая строчка, это уже будет не цикл. В "Си" для этого использовались скобочки, в Питоне от этого ушли, и тут "отступы".
# смысл цикла заключается в том, что нам нужно пройти по всем элементам списка (или "проитерироваться"),
# которые записаны поочередно, далее перейти к очередной части выборки (перейти к очередной паре соседних границ квантиля).
# цикл всегда "по умолчанию" идет вперед (от первого элемента к последнему), и ничего не пропускает.
# можно настроить и иначе, но нам этого не нужно тут.
for quantile in quantiles:
    # тут записывается граница квантиля, размер которого нами был задан выше.
    quantile_value = income.quantile(q=quantile)
    # назвали "что находится в квантиле". Почему тут надо ставить "больше-равно", "меньше равно", что иначе на полном ряде была бы нарушена логика.
    in_quantile = income[(income <= quantile_value) &  (income >= last_quantile_value)]    
    means_by_deciles.append((in_quantile.mean(), in_quantile.shape[0]))
    last_quantile_value = quantile_value
# это можно смотреть выше.
print(f"Средние доходы по децилям и количество наблюдений: {means_by_deciles}")

Количество тех, чей доход меньше дециля (17500.0): 498
Их средний доход: 11074.29718875502
Средние доходы по децилям и количество наблюдений: [(5818.9655172413795, 116), (10113.636363636364, 154), (12486.033519553073, 179), (14972.067039106149, 179), (17314.814814814814, 162), (19761.904761904763, 294), (24471.428571428572, 175), (29811.04651162791, 172), (37667.785234899326, 149)]


соберем всё в одну функцию, чтобы не повторяться, и сделаем то же самое, но сгруппировав наблюдения по типам населённых пунктов и местам проживания

In [67]:
# второй способ задать функцию через деф (от слова дефайн - определить).
# по сути, мы тут создали свой метод (функцию).
# можно обратить внимание, что функция выделяется отступами.
def describe(dataframe, column, title):
    print(title)
    print(dataframe[column].describe())
    
    variance = dataframe[column].var()
    print(f"Дисперсия: {variance}")
    
    skewness = dataframe[column].skew()
    print(f"Асимметрия: {skewness}")
    
    peakedness = dataframe[column].kurtosis()
    print(f"Эксцесс: {peakedness}")
    
    decil = dataframe[column].quantile(q=0.10)
    poorest = dataframe[dataframe[column] < decil]
    number_of_poorest = poorest.shape[0]
    print(f"Количество тех, чей доход меньше дециля ({decil}): {number_of_poorest}")
    mean = poorest[column].mean()
    print(f"Их средний доход: {mean}")
    
    income = dataframe[column]
    means_by_deciles = []
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    last_quantile_value = 0
    # дальше вложенный цикл (для каждого квантиля в квантилях).
    for quantile in quantiles:
        quantile_value = income.quantile(q=quantile)
        in_quantile = income[(income <= quantile_value) &  (income >= last_quantile_value)]
        means_by_deciles.append((in_quantile.mean(), in_quantile.shape[0]))
        last_quantile_value = quantile_value
    print(f"Средние доходы по децилям и количество наблюдений: {means_by_deciles}")
    
    fig = px.box(dataframe, y=column)
    fig.show()
    
    

In [68]:
column = 'Денежный доход в среднем на члена ДХ, в месяц, рублей'
for punct_type in yakutia['Тип населенного пункта'].unique(): # также можно сделать через groupby, 
    # но groupby медленный
    selected_by = yakutia[yakutia['Тип населенного пункта']==punct_type]
    title = f"{column}, {meta.value_labels[meta.variable_to_label['H00_04']][punct_type]}"
    describe(selected_by, column, title)

Денежный доход в среднем на члена ДХ, в месяц, рублей, Городской
count       669.000000
mean      26476.083707
std       17846.760205
min        3750.000000
25%       14583.333333
50%       22500.000000
75%       33750.000000
max      137500.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 318506849.80934995
Асимметрия: 1.9066438272033956
Эксцесс: 5.225483016320094
Количество тех, чей доход меньше дециля (11250.0): 65
Их средний доход: 7807.692307692308
Средние доходы по децилям и количество наблюдений: [(9119.047619047618, 105), (12231.182795698925, 93), (15208.333333333334, 96), (17272.727272727272, 88), (19868.42105263158, 171), (24362.745098039217, 102), (28218.75, 80), (32500.000000000004, 88), (42463.23529411765, 68)]


Денежный доход в среднем на члена ДХ, в месяц, рублей, Сельский
count       489.000000
mean      18128.834356
std       14216.347980
min        2500.000000
25%        9000.000000
50%       15000.000000
75%       22500.000000
max      137500.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 202104549.87775558
Асимметрия: 3.7029748336497232
Эксцесс: 22.04494127743238
Количество тех, чей доход меньше дециля (6428.571428571428): 45
Их средний доход: 4888.88888888889
Средние доходы по децилям и количество наблюдений: [(5096.153846153846, 52), (7363.636363636364, 55), (9805.194805194806, 77), (12761.627906976744, 86), (14179.104477611942, 67), (16966.29213483146, 89), (17705.479452054795, 73), (22008.928571428572, 56), (26675.0, 100)]


In [69]:
column = 'Денежный доход в среднем на члена ДХ, в месяц, рублей'
for punct_type in yakutia['Размер населенного пункта'].unique(): # также можно сделать через groupby, 
    # но groupby медленный
    selected_by = yakutia[yakutia['Размер населенного пункта']==punct_type]
    title = f"{column}, {meta.value_labels[meta.variable_to_label['R_1_1']][punct_type]}"
    describe(selected_by, column, title)

Денежный доход в среднем на члена ДХ, в месяц, рублей, городской,  менее 50,0 тыс.чел
count       448.000000
mean      27857.142857
std       18271.532687
min        3750.000000
25%       16875.000000
50%       22500.000000
75%       35000.000000
max      112500.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 333848906.7327155
Асимметрия: 1.6655698375976993
Эксцесс: 3.486698852412538
Количество тех, чей доход меньше дециля (11250.0): 30
Их средний доход: 6750.0
Средние доходы по децилям и количество наблюдений: [(8750.0, 54), (12539.0625, 64), (15932.20338983051, 59), (17214.285714285714, 70), (21130.13698630137, 73), (24049.295774647886, 71), (29493.243243243243, 74), (38321.91780821918, 73), (44734.04255319149, 47)]


Денежный доход в среднем на члена ДХ, в месяц, рублей, городской,  50,0 - 99,9 тыс.чел
count       74.000000
mean     18750.000000
std       9895.140867
min       8750.000000
25%      11250.000000
50%      14375.000000
75%      22343.750000
max      43750.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 97913812.78538814
Асимметрия: 0.950814277086461
Эксцесс: -0.2984259483347236
Количество тех, чей доход меньше дециля (9166.666666666666): 4
Их средний доход: 8750.0
Средние доходы по децилям и количество наблюдений: [(9038.461538461539, 13), (9821.428571428574, 14), (11153.846153846154, 13), (12368.421052631578, 19), (13750.0, 8), (16750.0, 10), (19833.333333333332, 15), (23653.846153846152, 13), (31666.666666666668, 12)]


Денежный доход в среднем на члена ДХ, в месяц, рублей, городской,  250,0 - 499,9 тыс.чел
count       147.000000
mean      26156.462585
std       18707.860811
min        5833.333333
25%       13750.000000
50%       21875.000000
75%       33750.000000
max      137500.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 349984056.12244904
Асимметрия: 2.3976879134693068
Эксцесс: 8.999543713036912
Количество тех, чей доход меньше дециля (9166.666666666666): 14
Их средний доход: 7500.0
Средние доходы по децилям и количество наблюдений: [(7794.117647058823, 17), (12100.0, 25), (14218.75, 16), (17421.875, 32), (19078.947368421053, 19), (22361.11111111111, 18), (25625.0, 32), (32031.25, 16), (40156.25, 16)]


Денежный доход в среднем на члена ДХ, в месяц, рублей, сельский,  более 5000 чел.
count      132.000000
mean     16988.636364
std      11752.342505
min       3500.000000
25%       7375.000000
50%      13750.000000
75%      22500.000000
max      55000.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 138117554.36039788
Асимметрия: 1.2592753144257316
Эксцесс: 1.0329824635937839
Количество тех, чей доход меньше дециля (5833.333333333333): 11
Их средний доход: 3636.3636363636365
Средние доходы по децилям и количество наблюдений: [(4624.999999999998, 20), (6323.529411764706, 17), (7391.304347826087, 23), (12321.42857142857, 28), (13750.0, 16), (16022.727272727272, 44), (17500.0, 20), (19285.714285714286, 35), (25952.380952380954, 21)]


Денежный доход в среднем на члена ДХ, в месяц, рублей, сельский,  201 - 1000 чел.
count      143.000000
mean     14353.146853
std       8033.989984
min       2500.000000
25%       7000.000000
50%      13500.000000
75%      17500.000000
max      35000.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 64544995.05775915
Асимметрия: 0.8372863211627785
Эксцесс: 0.04885768056909434
Количество тех, чей доход меньше дециля (5916.666666666667): 15
Их средний доход: 5000.0
Средние доходы по децилям и количество наблюдений: [(5000.0, 15), (6500.000000000002, 15), (7058.823529411765, 17), (8928.57142857143, 14), (12083.333333333334, 18), (14852.941176470587, 17), (17407.40740740741, 27), (17758.620689655174, 29), (23400.0, 25)]


Денежный доход в среднем на члена ДХ, в месяц, рублей, сельский,  1001 - 5000 чел.
count       214.000000
mean      21355.140187
std       17726.469740
min        4583.333333
25%       11354.166667
50%       16875.000000
75%       27500.000000
max      137500.000000
Name: Денежный доход в среднем на члена ДХ, в месяц, рублей, dtype: float64
Дисперсия: 314227729.4347601
Асимметрия: 3.713655593838277
Эксцесс: 17.623073201567696
Количество тех, чей доход меньше дециля (9000.0): 10
Их средний доход: 6250.0
Средние доходы по децилям и количество наблюдений: [(7900.0, 25), (9943.18181818182, 44), (12712.765957446809, 47), (14044.117647058823, 34), (15100.0, 25), (17600.0, 25), (21617.647058823528, 34), (24797.297297297297, 37), (29652.777777777777, 36)]
